In [1]:
#automatically reload stuff
%load_ext autoreload
%autoreload 2

In [2]:
#This is the scripts I have to import the data,
#save it as a intermediate json (patient_organ_data)
#then denoise that and save it as a second json (patient_organ_data_denoise)
#final version includes inputed values and the original mask

In [53]:
import Utils
import matplotlib.pyplot as plt
from OrganDataPreprocessing import CamprtOrganData
import pprint
import Formatting
from Constants import Const
import copy
import Cluster
import Models

In [20]:
Const.camprt_dir

'../data/CAMPRT_Centroids/'

In [10]:
#for the camprt data, centroids and distances are mapped to a folder for each patient
#where the folder name is the patient id
#this loads them and save them to a dict of the form {patient_id: {distances: distance_file, doses: doses_file}}
def load_spatial_files(root = None):
    #reads in the files for tumor centroids and ROI-tumor distances
    #returns {'id': {'distances': <distfile>, 'doses': <centroid/dosefile>}}
    #currently only returns patients with both ids
    root = Const.camprt_dir if root is None else root
    
    try:
        distance_files = glob(root + '**/*distances.csv')
    except:
        distance_files = []
    try:
        dose_files = glob(root + '**/*centroid*.csv')
    except: 
        dose_files = []
        
    def file_id(file):
        return max([int(x) for x in findall("[0-9]+", file)])
    
    dose_dict = {file_id(f): f for f in dose_files}
    dist_dict = {file_id(f): f for f in distance_files}
    
    dose_ids = set(dose_dict.keys())
    dist_ids = set(dist_dict.keys())
    shared_ids = dose_ids.intersection(dist_ids)
    
    #print which patients didn't have matches
    dropped_ids = dose_ids.symmetric_difference(dist_ids)
    if(len(dropped_ids) > 0):
        print("missing doses", dose_ids - shared_ids)
        print("missing distances", dist_ids - shared_ids)
        
    file_dict = {sid: {'doses': dose_dict.get(sid), 'distances': dist_dict.get(sid)} for sid in shared_ids}
    return file_dict

sfiles = load_spatial_files()
print('')
for k,v in sfiles.items():
    print('patient id',k)
    print('dose file',v['doses'])
    print('distance file',v['distances'])
    break

missing doses set()
missing distances {10153, 205, 10159, 281, 285}

patient id 3
dose file ../data/CAMPRT_Centroids/3/3_centroid.csv
distance file ../data/CAMPRT_Centroids/3/3_distances.csv


In [12]:
#dose files for this version are structured with coordinates, volume, and min/max/mean dose for each roi
#also not that the GTVXX are tumors, names are inconsistent for these and need cleaning
#GTVn is a nodal (lymph node) tumor, GTVp is a primary tumor
pd.read_csv(sfiles[3]['doses']).head()

,ROI,x coordinate,y coordinate,z coordinate,Structure Volume,Min Value,Mean Value,Max Value
0,GTV-N,295.3126,226.5868,77.6118,13.1,64.13,71.90,76.19
1,Brainstem,258.4537,245.9965,48.1683,23.5,1.52,12.60,39.70
2,Cricoid_cartilage,255.2230,233.9010,98.3618,4.5,8.12,29.76,46.31
3,Cricopharyngeal_Muscle,256.0970,241.5970,98.5522,1.0,11.60,30.93,43.08
4,Esophagus,260.1786,265.2096,119.2499,4.6,3.79,33.52,42.33


In [14]:
#Distances are paired ROI inter-region distances. 
#This is the minimum distance between the outer edges of the organ
#Note this is also annoying to work with
pd.read_csv(sfiles[3]['distances']).head()

,Reference ROI,Target ROI,Eucledian Distance (mm),Phi (degrees),Theta (degrees),% of Target Overlap,Eucledian Distance (mm) 5th Percentile
0,GTV-N,Brainstem,39.1372,11.0494,-157.0113,0.0,46.4944
1,GTV-N,Cricoid_cartilage,17.7550,-3.1798,180.0000,0.0,20.1082
2,GTV-N,Cricopharyngeal_Muscle,20.8533,-8.5625,-162.4744,0.0,23.4563
3,GTV-N,Esophagus,36.5443,-20.1214,-151.5571,0.0,43.1742
4,GTV-N,Extended_Oral_Cavity,15.7466,0.0000,150.2551,0.0,27.0794


In [28]:
#read in the organ info and save it to a default format

#In OrganDataPreprocessing I have a very complicated setup for processing a patient
#which includes a bunch of manual error checkers. See that code for reference, but new Dicom data will have different preprocessing
#In this print out, rename cols gives a list of things I renamed. You can see why I had to implement a generic spellchecking
od = CamprtOrganData()
pdict = od.process_cohort_spatial_dict(sfiles)
pdict.keys()
# Utils.np_dict_to_json(pdict,Const.processed_organ_json)
# del pdict

rename cols {'Sructure Volume': 'Structure Volume', 'Min value': 'Min Value'}
rename cols {'Mean Volume': 'Mean Value', 'Max Volume': 'Max Value'}
error reading patient 27
'<' not supported between instances of 'float' and 'str'
rename cols {'Mean Value ': 'Mean Value'}
rename cols {'Mean Value ': 'Mean Value'}
rename cols {'Max Vlaue': 'Max Value'}
rename cols {'min Value': 'Min Value'}
rename cols {'Max Vlaue': 'Max Value'}
rename cols {'Mean value': 'Mean Value'}
rename cols {'MinValue': 'Min Value'}
rename cols {'Mean value': 'Mean Value'}
rename cols {'Mean Dose': 'Mean doses'}
rename cols {'Mean_dose': 'Mean doses'}
rename cols {'mean dose': 'Mean doses'}
rename cols {'mean dose': 'Mean doses'}
rename cols {'mean dose': 'Mean doses'}
rename cols {'mean doses': 'Mean doses'}
rename cols {'mean doses': 'Mean doses'}
rename cols {'min Value': 'Min Value'}
rename cols {'Min Volume': 'Min Value', 'Mean Volume': 'Mean Value', 'Max Volume': 'Max Value'}
rename cols {'Mean value': 'Mean 

dict_keys(['organs', 'patients'])

In [32]:
#so this should check that the values make sense
#currently it implies there is an issue but I don't know what
def get_num_nans_per_organ(sdata):
    #should return a list of the % of patients missing data for each organ
    #for each entry type
    olist = sdata['organs']
    oarlist = olist + ['gtv']
    allnan = {}
    for key in ['distances','volume','centroids','mean_dose']:
        nancount = {o: 0 for o in oarlist}
        arr = Formatting.merged_spatial_array(sdata, key)
        for i,aa in enumerate(arr):
            #find missing organs for individual
            all_nan = np.argwhere(np.isnan(aa).all(axis=1))
            nanset = set([])
            if len(all_nan) < 1:
                continue
            for arg in all_nan[0]:
                nanorgan = oarlist[arg]
                nancount[nanorgan] = nancount[nanorgan] + 1
        nancount = [(k,np.round(100*nancount[k]/arr.shape[0],4)) for k,v in nancount.items() if v > 0]
        nancount = sorted(nancount, key = lambda x: -x[1])
        
        allnan[key] = nancount
    return allnan
    
get_num_nans_per_organ(pdict)

{'distances': [('Spinal_Cord', np.float64(1.81)),
  ('Rt_Brachial_Plexus', np.float64(0.905)),
  ('Thyroid_cartilage', np.float64(0.905)),
  ('Lt_Parotid_Gland', np.float64(0.905)),
  ('Rt_Ant_Digastric_M', np.float64(0.905)),
  ('Brainstem', np.float64(0.4525)),
  ('Rt_Parotid_Gland', np.float64(0.4525)),
  ('Rt_Lateral_Pterygoid_M', np.float64(0.4525)),
  ('Lt_Mastoid', np.float64(0.4525)),
  ('Hyoid_bone', np.float64(0.4525)),
  ('Glottic_Area', np.float64(0.4525))],
 'volume': [('Spinal_Cord', np.float64(1.81)),
  ('Esophagus', np.float64(0.905)),
  ('Rt_Brachial_Plexus', np.float64(0.905)),
  ('Thyroid_cartilage', np.float64(0.905)),
  ('Lt_Parotid_Gland', np.float64(0.905)),
  ('Rt_Ant_Digastric_M', np.float64(0.905)),
  ('Cricoid_cartilage', np.float64(0.4525)),
  ('Brainstem', np.float64(0.4525)),
  ('Rt_Parotid_Gland', np.float64(0.4525)),
  ('Rt_Lateral_Pterygoid_M', np.float64(0.4525)),
  ('Lt_Mastoid', np.float64(0.4525)),
  ('Hyoid_bone', np.float64(0.4525)),
  ('Glottic_A

In [37]:
#converts the json to a dictionary of arrays with all missing values imputed
#imputation here is the denosing autoencoder (see TSSIM paper), which improves results a bit and fills in missing data
#denoise alpha is the coefficient of how much of the denoising is done 
#using the autoencoder used to input missing values [0 = none, 1 = all]
di = Formatting.DataInputer(denoise_alpha = 0)
ddict = di.get_formatted_arrays(pdict,retrain=True)

#ddict is a dictionary of format {volume|distances|centroids|mean_dose : np.array}
#arrays are in teh shape n_patients, n_organs, n_channels
#channels are e.g. 3 for centroids (x,y,z), n_organs - 1 (no gtv distances) for distances, and 1 for volume and mean dose
[(k,v.shape) for k,v in ddict.items()]

training stopped on epoch 3349
OrganAutoEncoder(
  (hidden_layers): Sequential(
    (0): Linear(in_features=1845, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=20, bias=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Linear(in_features=20, out_features=100, bias=True)
    (9): ReLU()
    (10): Linear(in_features=100, out_features=200, bias=True)
    (11): ReLU()
    (12): Linear(in_features=200, out_features=1845, bias=True)
    (13): LeakyReLU(negative_slope=0.1)
  )
  (dropout_layer): Dropout(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)
tensor(0.1289, grad_fn=<MeanBackward0>)
volume
891.0 0.0 891.0 0.0
891.0 0.0 

distances
207.62500079518182 -24.640625 207.625 -24.640625
207.625 -24.640625 

centroids
658.3561 8.7422 658.3561 0.

[('volume', (221, 41, 1)),
 ('volume_missing', (221, 41, 1)),
 ('distances', (221, 41, 40)),
 ('distances_missing', (221, 41, 40)),
 ('centroids', (221, 41, 3)),
 ('centroids_missing', (221, 41, 3)),
 ('mean_dose', (221, 41, 1)),
 ('mean_dose_missing', (221, 41, 1))]

In [39]:
#check the error on the neural net for inputing/denoising
#only considers non-missing values
#value are before performing clipping on the inputed values
di.error_report()

,_mean_reconstruction_error (%),mean_denoised,std_denoised,shape_denoised,num_nan_denoised,num_negative_denoised,mean_original,std_original,shape_original,num_nan_original,num_negative_original
key,,,,,,,,,,,
volume,21.489422,18.24454,27.37419,"(221, 41, 1)",0,0,18.32490,27.51738,"(221, 41, 1)",131,0
distances,8.651396,42.63233,32.66252,"(221, 41, 40)",0,15540,42.49418,32.79505,"(221, 41, 40)",3506,16598
centroids,5.945143,173.50165,81.03168,"(221, 41, 3)",0,0,173.53767,81.73428,"(221, 41, 3)",141,0
mean_dose,60.317162,43.06255,17.94863,"(221, 41, 1)",0,0,42.48905,19.56367,"(221, 41, 1)",90,0


In [47]:
def extract_pdict_gtvs(p):
    gtvlist = []
    for pid,pentry in p['patients'].items():
        plist = []
        for o,oentry in pentry.items():
            if 'GTV' in o:
                plist.append((o,oentry))
        gtvlist.append((pid, plist))
    return gtvlist

#turn it into another dictionary again
def inputed_data_dict(pdata, insert_gtvs = True):
    pids = Formatting.get_sorted_pids(pdata)
    organs = pdata['organs'] + ['gtv_composite']
    main_keys = [k for k in ddict.keys() if '_missing' not in k]
    mask_keys =[k for k in ddict.keys() if k not in main_keys]
    make_dict = lambda keys: {int(p): {o: {k: np.nan for k in keys} for o in organs} for p in pids}
    inputed_data = make_dict(main_keys)
    data_mask = make_dict(mask_keys)
    for k,varray in ddict.items():
        for pid, v_row in zip(pids,varray):
            for organ, o_col in zip(organs, v_row):
                val = o_col
                if len(val) <= 1:
                    val = val[0]
                if k in mask_keys:
                    data_mask[int(pid)][organ][k.replace('_missing','')] = val
                else:
                    inputed_data[int(pid)][organ][k] = val
    return_dict = {'organs': organs, 'patient_ids': pids, 
            'patients': inputed_data, 'mask': data_mask}
    if insert_gtvs:
        gtvset = extract_pdict_gtvs(pdata)
        for pid,gtvs in gtvset:
            for k,v in gtvs:
                return_dict['patients'][int(pid)][k] = v
    return return_dict

#I'm going to be honest I have no idea what this does
pdata = inputed_data_dict(pdict)
[(k,v) for k,v in pdata['patients'][185].items() if 'GTV' in k]

[('GTVp',
  {'volume': np.float64(10.8),
   'mean_dose': np.float64(71.75),
   'centroids': array([279.6041, 198.22  ,  54.5221]),
   'distances': array([83.   , 27.97 , 41.62 , 82.5  , 59.4  , 53.4  , 35.6  , 20.47 ,
          29.28 , 26.66 , 27.72 , 55.06 , 72.2  , 53.78 , 34.56 , 44.6  ,
          57.3  , 18.39 , 36.75 , 12.84 , -3.973, -1.953, 12.16 , 19.34 ,
          11.59 , -4.98 , 35.94 , 10.05 , -4.027,  1.953, -3.32 , 28.08 ,
           2.5  ,  6.117, -3.906,  6.906, 23.94 , 58.1  , 58.62 , 49.88 ],
         dtype=float16)}),
 ('GTVn',
  {'volume': np.float64(4.6),
   'mean_dose': np.float64(72.28),
   'centroids': array([293.0174, 212.0342,  61.6571]),
   'distances': array([52.2   , 29.42  ,  6.734 , 63.9   , 30.45  , 29.8   , 14.49  ,
          14.945 , 43.4   , 20.02  , 11.66  , 59.22  , 89.7   , 66.7   ,
          61.5   , 72.56  , 78.7   , -1.953 , 29.8   ,  0.9766, -4.367 ,
          10.15  , 32.72  , 14.945 ,  9.34  ,  6.836 , 48.5   ,  7.875 ,
          26.11  , 21.3

In [46]:
#saves inputed_ddict
print(Const.denoised_organ_json)
# Utils.np_dict_to_json(pdata, Const.denoised_organ_json,True)

../data/patient_organ_data_denoised.json


In [48]:
def denoised_pdict_to_array(dpd,key,organ_list=None,pids=None):
    patients = dpd['patients']
    if organ_list is None:
        organ_list = dpd['organs']
    elif not Utils.iterable(organ_list):
        organ_list = [organ_list]
    if pids is None:
        pids = dpd['patient_ids']
    array = []
    for pid in pids:
        row = []
        pentry = patients.get(pid)
        for organ in organ_list:
            odata = pentry.get(organ)
            if odata is None:
                row.append(0)
            else:
                oval = odata.get(key)
                row.append(oval)
        array.append(row)
    return np.array(array)

#converts the dictionary to arrays to use for prediction
volumes = denoised_pdict_to_array(pdata,'volume')
distances = denoised_pdict_to_array(pdata,'distances')
denoised_pdict_to_array(pdata,'distances',organ_list =['gtv_composite'])

array([[[ 52.66879085,  25.79168075,  30.69005651, ...,  67.63982458,
          72.23524916,  30.47110926]],

       [[ 56.42200181,  28.43251164,  37.34030398, ...,  63.8253658 ,
          70.43469042,  35.31509627]],

       [[ 56.38310161,  37.12186748,  48.15841121, ...,  67.45791375,
          74.9777054 ,  37.44155612]],

       ...,

       [[ 26.8125    ,  24.578125  ,  13.328125  , ...,  73.125     ,
          79.6875    ,  14.5546875 ]],

       [[ 70.9375    ,  38.28125   ,  56.84375   , ..., 114.3125    ,
         127.6875    ,  58.09375   ]],

       [[ 65.125     ,  33.21875   ,  57.625     , ...,  78.1875    ,
          81.8125    ,  43.71875   ]]])

In [50]:
#the end results here is that for the Tssim code, we want a distances in the form n_patients, n_organs + gtv, n_organs
#and volumes of the form n_patients, n_organs + gtv
# where n_organs is defined in Const.organ_list, I think
volumes.shape, distances.shape

((221, 41), (221, 41, 40))

In [52]:
#takes in a dictionary of arrays 
#returns a similarity matrix 0-1
#n_jobs > 1 will try to multithread
sim = Models.TssimSimilarity(n_jobs = 4)
sim_matrix = sim.get_similarity_matrix(distances,volumes)
sim_matrix

array([[1.        , 0.68403329, 0.62034641, ..., 0.73218146, 0.61149602,
        0.59943214],
       [0.68403329, 1.        , 0.5765213 , ..., 0.59996719, 0.56147283,
        0.5637392 ],
       [0.62034641, 0.5765213 , 1.        , ..., 0.60720091, 0.59766111,
        0.60537348],
       ...,
       [0.73218146, 0.59996719, 0.60720091, ..., 1.        , 0.57902504,
        0.56704878],
       [0.61149602, 0.56147283, 0.59766111, ..., 0.57902504, 1.        ,
        0.70330457],
       [0.59943214, 0.5637392 , 0.60537348, ..., 0.56704878, 0.70330457,
        1.        ]])

In [54]:
#Similarity CLusterer does heirarchical clustering based on a similarity matrix, the method used in the TSSIM paper
#see https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html for what link does
#see https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.fcluster.html for what criterion does
clusters = Cluster.SimilarityClusterer(4,
                                       link='ward',#linkage method, passed to scipy.cluster.heirarchy.linkage
                                       criterion='maxclust',#criterion passed to scipy.cluster.heirarch.fcluster
                                      ).fit_predict(sim_matrix)
clusters

array([3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 2, 4, 4, 3, 4, 4, 3, 4,
       3, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 3, 2, 3, 2, 2, 3, 2, 3, 4, 3,
       2, 3, 3, 4, 3, 3, 3, 1, 3, 2, 2, 2, 2, 2, 2, 3, 1, 2, 3, 2, 2, 2,
       2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3,
       1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1,
       1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1], dtype=int32)

In [64]:
#We can also get the most similar patients using the TSSIM similarity matrix
#there are two match_type options, the normal knn (default) and the version in the camprt paper (threshold)
knn = Models.PatientKNN(
    match_type = 'default',#default used standard k-nearest neighbors
    default_n_matches=4,#number of neighbors is match_type = default
)
knn.get_matches(sim_matrix)[0:5]

[array([178, 158,  53, 148]),
 array([178,   0, 102,  41]),
 array([178, 175,  41, 102]),
 array([  0,   2, 178,   1]),
 array([178, 175, 119,  41])]

In [65]:
knn = Models.PatientKNN(
    match_type = 'threshold',#threshold takes a variable number of people
    match_threshold=.8, #if similarity is above this value, patient is considered a neighbor
    n_match_bounds=[1,3], #[min,max], lower and upper limit on the number of neighbors to return. 
)
knn_matches = knn.get_matches(sim_matrix)
[k for k in knn_matches if len(k) > 1][0:5]

[array([178, 158]),
 array([178, 148]),
 array([178, 102]),
 array([178, 175,  41]),
 array([178, 175,  41])]

In [78]:
#takes patient dictionary object and similarity matrix and returns a dictionary of neighbors, similarity scores, and clusters
#which are the data needed in a camprt style interface
#is similarity matrix is not passed, it computes it using tssim
def predict_patients(dpd,
                     sim = None, 
                     n_clusters = 4, #number of clusters to pass to SimilarityClusterer
                     return_dict= True,#whether to return a results dict or just the predictions. 
                     **knn_args,
                   ):
    if sim is None:
        volumes = denoised_pdict_to_array(pdata,'volume')
        distances = denoised_pdict_to_array(pdata,'distances')
        sim = Models.TssimSimilarity(n_jobs = 4).get_similarity_matrix(distances,volumes)
    pids = dpd['patient_ids']
    
    clusterer = Cluster.SimilarityClusterer(n_clusters)
    clusters = clusterer.fit_predict(sim)
    
    knn = Models.PatientKNN(**knn_args)
    knn_matches = knn.get_matches(sim_matrix)
    
    sim_dict = {}
    for i,pid in enumerate(pids):
        entry = {}
        entry['neighbors'] = [pids[ii] for ii in knn_matches[i]]
        entry['cluster'] = clusters[i]
        entry['similarity'] = sim[i,knn_matches[i]]
        sim_dict[pid] = entry
    if return_dict is False:
        thing = sorted([(k,v['cluster']) for k,v in sim_dict.items()],key=lambda x: x[0])
        return np.array([t[1] for t in thing])
    return sim_dict

#keyword arguments (besides sim) are passed to the PatientKnn constructure shown above
pp = predict_patients(pdata,sim_matrix,
                     match_type='threshold',
                     match_threshold=.8
                     )
for i,(patient_id, res) in enumerate(pp.items()):
    print('patient',patient_id)
    print(res)
    if i > 2:
        break

patient 3
{'neighbors': [10071, 10009], 'cluster': np.int32(3), 'similarity': array([0.82076472, 0.78579307])}
patient 4
{'neighbors': [10071], 'cluster': np.int32(4), 'similarity': array([0.69965742])}
patient 10
{'neighbors': [10071], 'cluster': np.int32(4), 'similarity': array([0.69498467])}
patient 11
{'neighbors': [3], 'cluster': np.int32(4), 'similarity': array([0.66117242])}


In [79]:
predict_patients(pdata,sim_matrix,
                     match_type='default',
                      return_dict=False
                     )[0:10]

array([3, 4, 4, 4, 4, 4, 4, 3, 4, 4], dtype=int32)